In [7]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
import os.path
import json

import time
import logging
from datetime import datetime

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# If modifying these scopes, delete the file token.json.
SCOPES = ["https://www.googleapis.com/auth/gmail.modify"]

load_dotenv(find_dotenv())
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
client = OpenAI(api_key=OPENAI_API_KEY)

In [8]:
creds = None
  # The file token.json stores the user's access and refresh tokens, and is
  # created automatically when the authorization flow completes for the first
  # time.
""" if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json", SCOPES) """
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            "credentials.json", SCOPES
        )
        creds = flow.run_local_server(port=4000)
    # Save the credentials for the next run
    with open("token.json", "w") as token:
        token.write(creds.to_json())

service = None
try:
    # Call the Gmail API
    service = build("gmail", "v1", credentials=creds)

except HttpError as error:
# TODO(developer) - Handle errors from gmail API.
    print(f"An error occurred: {error}")

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=915927531429-jb5jkim9325sqr9bun1hvo1evtc961rq.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A4000%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fgmail.modify&state=eldTASM4CZxOQVYOaK09x1wntuAV21&access_type=offline


In [9]:
#Gmail API Calls
def get_email(id):
    try:
        result = service.users().messages().get(userId='me', id=id).execute()

        if not result:
            print("Message not found.")
            return
        print("Message:" + result)
        return result

    except HttpError as error:
    # TODO(developer) - Handle errors from gmail API.
        print(f"An error occurred: {error}")

def get_email_list():
    try:
        results = service.users().messages().list(userId="me").execute()
        messages = results.get("messages", [])

        if not messages:
            print("No messages found.")
            return
        print("Message:")
        for message in messages:
            print(message)
        return messages

    except HttpError as error:
    # TODO(developer) - Handle errors from gmail API.
        print(f"An error occurred: {error}")


def send_email(content, to, sender, subject):
  """Create and send an email message
  Returns: Message object, including message id

  Load pre-authorized user credentials from the environment.
  """

  try:
    message = EmailMessage()

    message.set_content(content)

    message["To"] = to
    message["From"] = sender
    message["Subject"] = subject

    # encoded message
    encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()

    create_message = {"raw": encoded_message}
    # pylint: disable=E1101
    send_message = (
        service.users()
        .messages()
        .send(userId="me", body=create_message)
        .execute()
    )
    return send_message
  except HttpError as error:
    print(f"An error occurred: {error}")
    send_message = None

send_email("Hello", to, sender, subject)
    

In [10]:
# Assistant class that can make Gmail API calls
class GmailHelper:
    thread_id = None
    assistant_id = "asst_p01Djp64FjuGUIX9kf72DyTe"

    def __init__(self):
        self.client = client
        self.model = "gpt-4"
        self.assistant = None
        self.thread = None
        self.run = None

        if GmailHelper.assistant_id:
            self.assistant = self.client.beta.assistants.retrieve(assistant_id=GmailHelper.assistant_id)

        if GmailHelper.thread_id:
            self.thread = self.client.beta.threads.retrieve(thread_id=GmailHelper.thread_id)

    def create_assistant(self, name, instructions, tools):
        if not self.assistant:
            assistant_obj = self.client.beta.assistants.create(
                name=name,
                instructions=instructions,
                tools=tools,
                model=self.model
            )
            GmailHelper.assistant_id = assistant_obj.id
            self.assistant = assistant_obj
            print(f"Assistant created with ID: {self.assistant.id}")

    def create_thread(self):
        if not self.thread:
            thread_obj = self.client.beta.threads.create()
            GmailHelper.thread_id = thread_obj.id
            self.thread = thread_obj
            print(f"Thread created with ID: {self.thread.id}")

    def add_message_to_thread(self, role, content):
        if self.thread:
            self.client.beta.threads.messages.create(
                thread_id = self.thread.id,
                role = role,
                content = content
            )
    
    def run_assistant(self, instructions):
        if self.thread and self.assistant:
            self.run = self.client.beta.threads.runs.create(
                thread_id = self.thread_id,
                assistant_id = self.assistant_id,
                instructions = instructions
            )

    def print_thread(self):
        if self.thread:
            messages = self.client.beta.threads.messages.list(thread_id = self.thread_id)

            for message in messages:
                role = message.role
                content = message.content[0].text.value
                print(role + ': ' + content)

    def call_required_functions(self, required_actions):
        if not self.run:
            return
        tool_outputs = []

        for action in required_actions["tool_calls"]:
            func_name = action["function"]["name"]
            arguments = json.loads(action["function"]["arguments"])

            if func_name == "get_email_list":
                output = get_email_list()
            elif func_name == "get_email":
                output = get_email(id=arguments["id"])
            elif func_name == "send_email":
                output = send_email(content=arguments["content"], to=arguments["to"], sender=arguments["sender"], subject=arguments["subject"])

            final_str = ""
            for item in output:
                final_str += "".join(item)

            print(f"Tool output: {output}")
            tool_outputs.append({"tool_call_id": action["id"], "output": final_str})

        self.client.beta.threads.runs.submit_tool_outputs(
            thread_id = self.thread.id,
            run_id = self.run.id,
            tool_outputs = tool_outputs
        )
    
    def wait_for_completion(self):
        if self.thread and self.run:
            while True:
                time.sleep(5)
                run_status = self.client.beta.threads.runs.retrieve(
                    thread_id = self.thread.id,
                    run_id = self.run.id
                )

                if run_status.status == "completed":
                    break
                elif run_status.status == "requires_action":
                    self.call_required_functions(required_actions=run_status.required_action.submit_tool_outputs.model_dump())

    def run_steps(self):
        run_steps = self.client.beta.threads.runs.steps.list(
            thread_id = self.thread.id,
            run_id = self.run.id
        )
        print(f"Run steps: {run_steps}")
        return run_steps.data


In [11]:
manager = GmailHelper()

In [12]:
manager.create_assistant(
    name = "Gmail Helper",
    instructions = "Your go-to assistant for reading and editing emails. Gmail Helper is a personal assistant dedicated to reading and sending emails, primarily through Google Gmail API integration. Its core functionality includes checking the user’s email to read and summarize emails they receive. It is capable of sending emails to the user's contacts but will always seek confirmation from the user before doing so, ensuring that the user has control over what they send. Gmail Helper maintains a professional demeanor, focusing on providing clear, concise, and accurate scheduling information. It prioritizes user privacy and confidentiality.The assistant is also equipped to guide users through the Google Gmail API’s functionalities and troubleshoot common issues, using language tailored to the user’s level of technical expertise. Overall, Gmail Helper adopts a polite, efficient, and accommodating tone, aiming to make email management and event scheduling as smooth and user-friendly as possible.",
    tools = [{
      "type": "function",
      "function": {
        "name": "get_email_list",
        "description": "Get a list of the user's mailbox contents",
      }
    }, {
      "type": "function",
      "function": {
        "name": "get_email",
        "description": "Get the contents of an email",
        "parameters": {
          "type": "object",
          "properties": {
              "id": {
                  "type": "string",
                  "description": "The ID of the email to be retrieved",
              },
          },
          "required": ["id"],
        }
      }
    }, {
      "type": "function",
      "function": {
          "name": "send_email",
        "description": "Send an email to another user",
        "parameters": {
        "type": "object",
        "properties": {
          "content": {"type": "string", "description": "The content of the email to be sent"},
          "to": {"type": "string", "description": "Email address of email recipient"},
          "sender": {"type": "string", "description": "The sender of the email, namely the user"},
          "subject": {"type": "string", "description": "The subject of the email being sent"}
        },
        "required": ["content", "to", "sender", "subject"]
        }
      }
  }]
)

manager.create_thread()

manager.add_message_to_thread(
    role = "user",
    content = "What are my latest emails?"
)

manager.run_assistant(instructions="Get the user's latest emails")
manager.wait_for_completion()

manager.print_thread()

Thread created with ID: thread_0dqGMLUHNnkedXBjtxE517Xc
Message:
{'id': '18dfb66655c0d3eb', 'threadId': '18dfb58257b9bfa3'}
{'id': '18dfb58257b9bfa3', 'threadId': '18dfb58257b9bfa3'}
{'id': '18d8fe5fc76d5fc6', 'threadId': '18d8fe5fc76d5fc6'}
{'id': '18d621c155488a65', 'threadId': '18d621c155488a65'}
{'id': '18d47366e5e864e4', 'threadId': '18d47008328f455b'}
{'id': '18d4706eadc8fefc', 'threadId': '18d4706eadc8fefc'}
{'id': '18d47008328f455b', 'threadId': '18d47008328f455b'}
{'id': '18d3e1d1548834f7', 'threadId': '18d3ddb334e03e9b'}
{'id': '18d3ddb334e03e9b', 'threadId': '18d3ddb334e03e9b'}
{'id': '18d2fe6b09b8b23e', 'threadId': '18d2fe6b09b8b23e'}
{'id': '18d016cfb875d28c', 'threadId': '18d016b9b188814f'}
{'id': '18d016b9b188814f', 'threadId': '18d016b9b188814f'}
{'id': '18c4732a4942eb0d', 'threadId': '18c4731ff96beaba'}
{'id': '18c472f9cc9725a4', 'threadId': '18c472f9cc9725a4'}
{'id': '18c10d462c705878', 'threadId': '18c10d462c705878'}
{'id': '18bb0bbf3da9d381', 'threadId': '18bb0bbf3d

In [13]:
manager.add_message_to_thread(
    role = "user",
    content = "Show me the content of the email of the first."
)

manager.run_assistant(instructions="Tell the user what the content of the email is")
manager.wait_for_completion()
manager.print_thread()

An error occurred: <HttpError 400 when requesting https://gmail.googleapis.com/gmail/v1/users/me/messages/idthreadId1?alt=json returned "Invalid id value". Details: "[{'message': 'Invalid id value', 'domain': 'global', 'reason': 'invalidArgument'}]">


TypeError: 'NoneType' object is not iterable

In [17]:
service.close()